# SEO Notebook

In [1]:
"""A framework for conducting SEO investigations, using Google Spreadsheets for I/O."""

'A framework for conducting SEO investigations, using Google Spreadsheets for I/O.'

In [2]:
import notebook_finder    #Allows .ipynb files to be loaded like filename.py modules
import goodsheet          #Handles OAuth login for all relevant Google services
from imp import reload    #Keeps in-memory functions reflecting your pipulate.ipynb edits
import shelve, requests

In [3]:
cache_html = True
google_sheet_name = 'Lookups'
google_sheet_tab  = 'Sheet1'
rows_to_batch = 5

In [4]:
if __name__ == '__main__':
    import pipulate
    reload(pipulate)
    import pipulate #intentional
    worksheet = goodsheet.connect.open(google_sheet_name).worksheet(google_sheet_tab)
    pipulate_funcs = [x for x in dir(pipulate) if x[0] is not '_']
    func_dict = {x.lower():eval('pipulate.%s' % x) for x in pipulate_funcs}
    try:
        rows = worksheet.row_count
        cols = worksheet.col_count
        end_range = worksheet.get_addr_int(rows, cols)
        #cell_range = worksheet.range('A1:%s' % end_range)
        col_count = worksheet.col_count
    except:
        print("Cannot reach Google Sheets.")
        raise SystemExit()
    
    chunk_ranges = [(x+1, x+rows_to_batch) for x in list(range(rows)) if x%rows_to_batch == 0]        
    for i, (row_start, row_end) in enumerate(chunk_ranges):
        top_left = worksheet.get_addr_int(row_start+1, 1)
        lower_right = worksheet.get_addr_int(row_end+1, cols)
        range_string = "%s:%s" % (top_left, lower_right)
        print(i+1, len(chunk_ranges))
        if i+1 == len(chunk_ranges):
            print("hit")
            range_string = "%s:%s" % (top_left, end_range)
        print(range_string)
        cell_range = worksheet.range(range_string)

    raise SystemExit()

    col_names = []
    row_dict = {}
    print("Pipulating row: ", end="")

    
    for i, acell in enumerate(cell_range):
        row, col, val = acell.row, acell.col, acell.value
        if (col)%col_count == 0:
            print(row, end=", ")
        if row == 1:
            col_names.append(val.lower())
        else:
            row_dict[col_names[col-1]] = val
            if col%col_count == 0:
                # Reached end of row, let's Pipulate!
                response = None
                if 'url' in row_dict:
                    with shelve.open('urls') as urls:
                        if cache_html == True and row_dict['url'] in urls.keys():
                            response = urls[row_dict['url']]
                        else:
                            try:
                                response = requests.get(row_dict['url'])
                                urls[row_dict['url']] = response
                            except requests.exceptions.RequestException as e:
                                raise SystemExit()
                        row_dict['response'] = response
                for key, val in row_dict.items():
                    if not val:
                        if key in [x.lower() for x in dir(pipulate) if x[0] is not '_']:
                            success_code, new_text = func_dict[key](**row_dict)
                            cell_address = (row*col_count) - col_count + col_names.index(key)
                            cell_range[cell_address].value = new_text
                row_dict = {}
    print("Done, updating spreadsheet...")
    try:
        worksheet.update_cells(cell_range)
        print("Updated!")
    except:
        print("Update failed.")

0 9
A2:B6
1 9
A7:B11
2 9
A12:B16
3 9
A17:B21
4 9
A22:B26
5 9
A27:B31
6 9
A32:B36
7 9
A37:B41


HTTPError: 400: b'Invalid query parameter value for range.'